In [1]:
import clip

In [15]:
from sklearn.model_selection import train_test_split
import torch
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from clip import CLIPModel

# Load CSV file
csv_path = 'cat_dog1.csv'
df = pd.read_csv(csv_path)

# Assuming your CSV has 'image' and 'labels' columns
images = df['image'].values
labels = df['labels'].values

# Split the dataset
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Load the CLIP model using a deep learning framework like PyTorch
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("ViT-B/32", device=device)

# Utilize the CLIP model to extract embeddings for each image in the dataset (both train and test dataset)
train_embeddings = []
test_embeddings = []

for image_path, label in zip(train_images, train_labels):
    # Load image using PIL
    image = model.preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = model.encode_image(image)
    train_embeddings.append(embedding.cpu().numpy())

for image_path, label in zip(test_images, test_labels):
    # Load image using PIL
    image = model.preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = model.encode_image(image)
    test_embeddings.append(embedding.cpu().numpy())

# Organize the extracted CLIP embeddings into a feature matrix
train_features = np.concatenate(train_embeddings)
test_features = np.concatenate(test_embeddings)

# Fisher Algorithm
C_values = [0.1, 1.0, 10.0]
best_accuracy = 0.0
best_C = None


ImportError: cannot import name 'CLIPModel' from 'clip' (c:\Python311\Lib\site-packages\clip\__init__.py)

In [ ]:
for C in C_values:
    fisher_model = LinearDiscriminantAnalysis(solver='eigen', shrinkage=C)
    fisher_model.fit(train_features, train_labels)

    # Test the model using the testing dataset and visualize the classification results using a confusion matrix
    predictions = fisher_model.predict(test_features)
    confusion_mat = confusion_matrix(test_labels, predictions)

    plt.imshow(confusion_mat, cmap='Blues')
    plt.colorbar()
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(f'Confusion Matrix for C={C}')
    plt.show()

    # Model Evaluation
    accuracy = accuracy_score(test_labels, predictions)
    precision = precision_score(test_labels, predictions)
    recall = recall_score(test_labels, predictions)
    f1 = f1_score(test_labels, predictions)

    print(f"\nMetrics for C={C}:")
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-score:", f1)

    # Check if the current model is the best based on accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_C = C

print(f"\nBest C value: {best_C} with Accuracy: {best_accuracy}")